In [46]:
import pandas as pd
 


In [47]:
 
#所需的欄位
#序號,縣市別代碼,機關代碼,市話,傳真,年,月,日,時,分,縣市,區,死亡數量,受傷數量,2-30,天候,光線,道路類別,速限,道路型態,
#事故位置,路面鋪裝,路面狀態,路面缺陷,障礙物,視距,號誌種類,號誌動作,分向設施,快車道或一般車道間,快慢車道間,路面邊線,
#事故類型及型態,主要肇因,受傷程度,主要傷處,保護裝備,行動電話,當事者區分(類別),車輛用途,當事者行動狀態,駕駛資格情形,
#駕駛執照種類,飲酒情形,車輛撞擊部位最初,車輛撞擊部位其他,肇事因素個別,肇事因素主要,肇事逃逸,職業,旅次目的,車種,GPS座標X,GPS座標Y,事故類別
usecols = ['序號', '年', '月', '日', '時', '區', '天候', '死亡數量' , '受傷數量', '飲酒情形', '肇事因素個別', '肇事因素主要', '事故類型及型態', '事故類別', 'GPS座標X', 'GPS座標Y']
df = pd.read_csv('rawdata/11207.csv', usecols=usecols)
df['date'] = df['日'].map('2023-07-{:02d}'.format)


In [88]:
df

,序號,年,月,日,時,區,死亡數量,受傷數量,天候,事故類型及型態,飲酒情形,肇事因素個別,肇事因素主要,GPS座標X,GPS座標Y,事故類別,date
0,1,2023,7,1,0,中區,0,1,7.0,13.0,2.0,7.0,7.0,120.678612,24.143256,2,2023-07-01
1,2,2023,7,1,0,北區,0,0,8.0,NaN,2.0,8.0,NaN,120.676711,24.150642,3,2023-07-01
2,3,2023,7,1,0,太平區,0,0,8.0,27.0,2.0,49.0,NaN,120.721744,24.154998,3,2023-07-01
3,4,2023,7,1,1,梧棲區,0,0,7.0,NaN,2.0,84.0,84.0,120.534472,24.263000,3,2023-07-01
4,5,2023,7,1,1,西屯區,0,0,7.0,17.0,10.0,43.0,43.0,120.614213,24.165561,3,2023-07-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12169,12170,2023,7,31,23,北屯區,0,1,7.0,16.0,2.0,26.0,26.0,120.685498,24.172177,2,2023-07-31
12170,12171,2023,7,31,23,西屯區,0,0,7.0,NaN,2.0,49.0,NaN,120.614006,24.179447,3,2023-07-31
12171,12172,2023,7,31,23,北屯區,0,1,7.0,17.0,2.0,84.0,84.0,120.690377,24.176862,2,2023-07-31
12172,12173,2023,7,31,23,大雅區,0,0,7.0,NaN,2.0,20.0,NaN,120.631697,24.240028,3,2023-07-31


In [50]:
df['date']

0        0         1\n1         1\n2         1\n3      ...
1        0         1\n1         1\n2         1\n3      ...
2        0         1\n1         1\n2         1\n3      ...
3        0         1\n1         1\n2         1\n3      ...
4        0         1\n1         1\n2         1\n3      ...
                               ...                        
12169    0         1\n1         1\n2         1\n3      ...
12170    0         1\n1         1\n2         1\n3      ...
12171    0         1\n1         1\n2         1\n3      ...
12172    0         1\n1         1\n2         1\n3      ...
12173    0         1\n1         1\n2         1\n3      ...
Name: date, Length: 12174, dtype: object

In [33]:
# 設定
area_filter = df['區']=='西屯區'

In [45]:
df[(df['區']=='清水區')][['GPS座標X', 'GPS座標Y']].rename(columns = {'GPS座標X':'x', 'GPS座標Y':'y'}).head()


,x,y
9,120.571132,24.259066
120,120.564121,24.269610
124,120.537777,24.273255
217,120.566296,24.268445
247,120.568180,24.271821


In [34]:
# 台中各區事故總量
df.groupby('區')['序號'].count().reset_index(name='count').head()

,區,count
0,中區,152
1,北區,802
2,北屯區,1256
3,南區,537
4,南屯區,974


In [40]:
# 某區事故座標
df[area_filter][['GPS座標X', 'GPS座標Y']].rename(columns = {'GPS座標X':'x', 'GPS座標Y':'y'}).head()



,x,y
4,120.614213,24.165561
12,120.636056,24.184583
14,120.655471,24.197625
25,120.642669,24.182436
34,120.612373,24.184395


In [36]:
# 事故類型統計詞雲
df.groupby('事故類型及型態')['序號'].count().reset_index(name='count').head()

,事故類型及型態,count
0,1.0,24
1,2.0,64
2,3.0,115
3,4.0,2
4,5.0,1


In [37]:
# 天候與事故的關聯
df.groupby('天候').apply(func=lambda df_gr: pd.Series({
    ('事故總數'): len(df_gr),
    ('發生天數'): len(df_gr.groupby('日')),
    ('每日事故平均數'): len(df_gr)/len(df_gr.groupby('日')),
    ('每日事故中位數'): df_gr.groupby('日')['序號'].count().reset_index(name='count')['count'].median(),
    ('單日最大事故數'): df_gr.groupby('日')['序號'].count().reset_index(name='count').sort_values('count', ascending=False).iloc[0]['count'],
    ('最多事故日'): df_gr.groupby('日')['序號'].count().reset_index(name='count').sort_values('count', ascending=False).iloc[0]['日'],
    }))#.astype({'事故總數': 'int', '發生天數': 'int', '每日事故平均數': 'int'})

,事故總數,發生天數,每日事故平均數,每日事故中位數,單日最大事故數,最多事故日
天候,,,,,,
1.0,4.0,4.0,1.000000,1.0,1.0,6.0
4.0,1.0,1.0,1.000000,1.0,1.0,28.0
5.0,1191.0,21.0,56.714286,26.0,391.0,28.0
6.0,489.0,31.0,15.774194,8.0,83.0,27.0
7.0,9604.0,31.0,309.806452,328.0,511.0,14.0
8.0,883.0,12.0,73.583333,80.0,111.0,3.0


In [39]:
# 各區每日事故量
df.groupby(['區', '日'])['序號'].count().reset_index(name='count').head()

,區,日,count
0,中區,1,7
1,中區,2,7
2,中區,4,5
3,中區,5,5
4,中區,6,5


In [90]:
# 尖峰時刻
df.groupby(['時'])['序號'].count().reset_index(name='count')#.head()

,時,count
0,0,108
1,1,63
2,2,40
3,3,41
4,4,43
5,5,53
6,6,173
7,7,742
8,8,919
9,9,705
